In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


import datetime, time
import numpy as np
import pandas as pd
from scipy import stats
import os
import subprocess

In [ ]:
states = 5
days = 10

sp = np.genfromtxt(os.path.join('./sims1', 'companies_final.csv'), delimiter=',', dtype=None)
tickers = sp[:,0]
t = tickers[0:]
samples = t.size
m=np.zeros(samples)
r=np.zeros(samples)
v=np.zeros(samples)
p=np.zeros(samples)
l=np.zeros(samples)
ml=np.zeros(samples)

In [ ]:

print (t)

In [ ]:
i=0
pd.options.display.float_format = '{:,.4f}'.format
for company in t:
    fname = company +"_"+ str(states) + "_" + str(days) + "_kdes.csv"
    dfrtmp = pd.read_csv(os.path.join('./sims1', fname))
    #print (dfrtmp)
    #print(dfrtmp.at[0,'E']) #MLS
    m[i] = dfrtmp.at[0,'E']
    #print(dfrtmp.at[1,'E']) #RAND
    r[i] = dfrtmp.at[1,'E']
    #print(dfrtmp.at[2,'E']) #MLS
    v[i] = dfrtmp.at[2,'E']
    #print(dfrtmp.at[3,'E']) #MLS
    p[i] = dfrtmp.at[3,'E']
    #print(dfrtmp.at[4,'E']) #MLS
    l[i] = dfrtmp.at[4,'E']
    #print(dfrtmp.at[5,'E']) #MLS
    ml[i] = dfrtmp.at[5,'E']
    i=i+1

In [ ]:
dfr = pd.DataFrame({  #"MLS", "Rand","Vtr","Psd","Lls","Mlns
                    'MLS' : m,
                    'RAND' : r,
                    'VTR' : v,
                    'PSD' : p,
                    'LLS' : l,
                    'MLNS' : ml,        
                     })

In [ ]:
dfr.tail(3)

In [ ]:
dfr.describe()[:3]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(9, 6))
data = [l,ml,m,p,r,v]
plt.boxplot(data, 0, '',widths=0.75);
plt.xticks([1, 2, 3,4,5,6], ['LLS', 'MLNS', 'MLS', 'PSD', 'RAND', 'VTR']);
plt.ylabel("Gaussian KDE @real price")
plt.savefig('boxplot_starting_states.png', bbox_inches='tight')

In [ ]:
d1 = dfr['MLS']
d2 = dfr['RAND']
d3 = dfr['LLS']
d4 = dfr['PSD']
d5 = dfr['VTR']
d6 = dfr['MLNS']

In [ ]:
#z_stat, p_val = stats.ranksums(d1,d2)

In [ ]:
#print (p_val)

In [ ]:
p_val1 = stats.wilcoxon(d1,d2, zero_method='wilcox', correction=False)[1]
p_val2 = stats.wilcoxon(d1,d3, zero_method='wilcox', correction=False)[1]
p_val3 = stats.wilcoxon(d1,d4, zero_method='wilcox', correction=False)[1]
p_val4 = stats.wilcoxon(d1,d5, zero_method='wilcox', correction=False)[1]
p_val5 = stats.wilcoxon(d1,d6, zero_method='wilcox', correction=False)[1]

In [ ]:
print("P-value comparison between MLS and RAND: ", p_val1) # Used because data is paired  :D
print("P-value comparison between MLS and LLS: ", p_val2)
print("P-value comparison between MLS and PSD: ", p_val3)
print("P-value comparison between MLS and VTR: ", p_val4)
print("P-value comparison between MLS and MLNS: ", p_val5)

In [ ]:
filename = "Table_Comparison_" + str(states) + "_States_" + str(days) + "Days" + '.tex'

template = r'''\documentclass[preview]{{standalone}}
\usepackage{{booktabs}}
\begin{{document}}
{}
\end{{document}}
'''

with open(filename, 'wb') as f:
    f.write(template.format(dfr.describe()[:3].to_latex()))

subprocess.call(['pdflatex', filename])

In [ ]:
pd.options.display.float_format = '{:,.6f}'.format
dfr_pvalues =  pd.DataFrame({       
                    '': pd.Categorical(["MLS vs."]),
                    'RAND':p_val1,
                    'LLS':p_val2,
                    'PSD':p_val3,
                    'VTR':p_val4,
                    'MLNS':p_val5,
                    
                     }, index=['p-value'])
#'D':np.array([0] * 2,dtype='int32'),

In [ ]:
dfr_pvalues

In [ ]:
filename = "Pvalues_Comparison_" + str(states) + "_States_" + str(days) + "Days" + '.tex'

template = r'''\documentclass[preview]{{standalone}}
\usepackage{{booktabs}}
\begin{{document}}
{}
\end{{document}}
'''

with open(filename, 'wb') as f:
    f.write(template.format(dfr_pvalues.to_latex()))

subprocess.call(['pdflatex', filename])